In [3]:
#import tensorflow, sklearn, matplotlib, numpy and pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import requests
import json
import pywhatkit

from datetime import timedelta
from matplotlib import pyplot
from pickle import dump

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model

from datetime import datetime

from sklearn.metrics import mean_squared_error

scaler = MinMaxScaler()

#configure pandas to show the full dataframe
pd.set_option('display.max_columns', None)

In [4]:
#write a function that takes input in radians and returns sin + cos of that input
def sin_cos(x, mode):
    denom = 2*math.pi

    if mode == 'hour':
        return math.sin((int(x)/24)*denom) + math.cos((int(x)/24)*denom)
    elif mode == 'day':
        return math.sin((int(x)/31)*denom) + math.cos((int(x)/31)*denom)
    else:
        return math.sin((int(x)/12)*denom) + math.cos((int(x)/12)*denom)


In [5]:
#write a function to merge dataset_AQI and dataset_Weather on their indices
def merge_datasets(dataset_AQI, dataset_Weather):
    #join the two dataframes on their indices
    dataset_merged = dataset_AQI.join(dataset_Weather, how='inner')
    #dataset_AQI.merge(dataset_Weather, left_index=True, right_index=True)
    return dataset_merged

In [6]:
#make a function that takes 12 hour time and converts it to 24 hour time
def convert24(str1):

    str1 = str(str1)
      
    # Checking if last two elements of time
    # is AM and first two elements are 12
    if str1[-2:] == "AM" and str1[:2] == "12":
        return "00" + str1[2:-2]
          
    # remove the AM    
    elif str1[-2:] == "AM":
        return str1[:-2]
      
    # Checking if last two elements of time
    # is PM and first two elements are 12   
    elif str1[-2:] == "PM" and str1[:2] == "12":
        return str1[:-2]
          
    else:  
        # add 12 to hours and remove PM
        return str(int(str1[:2]) + 12) + str1[2:5]

In [7]:
#write a function that converts degrees to radians
def deg2rad(deg):
    return math.sin(deg * (math.pi/180)) + math.cos(deg * (math.pi/180))

In [8]:
#import dataset city_hour.csv into a dataframe dataset
dataset_AQI = pd.read_csv('city_hour.csv')

#import dataset delhi.csv into a dataframe with date_time as index
dataset_Weather = pd.read_csv('delhi.csv')

In [9]:
#fill NaN values in all columns with interpolation
dataset_AQI['PM2.5'] = dataset_AQI['PM2.5'].interpolate()

#remove rows where all columns are NaN
dataset_AQI = dataset_AQI.dropna()

#give the number of NaN values in dataset
dataset_AQI.isnull().sum()

#give size of dataset_AQI
dataset_AQI.shape

#make a new dataframe where the City column is 'Gurugram'
dataset_AQI = dataset_AQI[dataset_AQI['City'] == 'Delhi']

#rename dataset_Weather column date_time to Datetime
dataset_Weather.rename(columns={'date_time':'Datetime'}, inplace=True)

In [10]:
#set index of dataset_Weather to Datetime
dataset_Weather.set_index('Datetime', inplace=True)
dataset_Weather.index = dataset_Weather.index.astype(str)

#set index of dataset_AQI to Datetime
dataset_AQI.set_index('Datetime', inplace=True)
dataset_AQI.index = dataset_AQI.index.astype(str)

In [11]:
#merge the two datasets
dataset_AQI_Weather = merge_datasets(dataset_Weather, dataset_AQI)

In [12]:
rem_columns = ['totalSnow_cm', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3','Benzene', 'Toluene', 'Xylene', 'AQI_Bucket', 'City', 'moonrise', 'moonset', 'moon_illumination', 'HeatIndexC', 'WindChillC', 'uvIndex']
#drop columns from dataset_AQI_Weather that are in rem_columns
dataset_AQI_Weather.drop(rem_columns, axis=1, inplace=True)

#remove all space in columns 'sunrise', 'sunset'
dataset_AQI_Weather['sunrise'] = dataset_AQI_Weather['sunrise'].apply(lambda x: x.replace(" ",""))
dataset_AQI_Weather['sunset'] = dataset_AQI_Weather['sunset'].apply(lambda x: x.replace(" ",""))

#Convert columns 'sunrise', 'sunset' using convert24 funnction
dataset_AQI_Weather['sunrise'] = dataset_AQI_Weather['sunrise'].apply(convert24)
dataset_AQI_Weather['sunset'] = dataset_AQI_Weather['sunset'].apply(convert24)

#convert 'sunrise', 'sunset' to seconds
dataset_AQI_Weather['sunrise'] = dataset_AQI_Weather['sunrise'].apply(lambda x: (int(x[:2])*3600 + int(x[3:5])*60)/86400)
dataset_AQI_Weather['sunset'] = dataset_AQI_Weather['sunset'].apply(lambda x: (int(x[:2])*3600 + int(x[3:5])*60)/86400)

#make new column Datetime from index
dataset_AQI_Weather['Datetime'] = dataset_AQI_Weather.index

#make columns 'hour', 'day', 'month', 'year' from Datetime
dataset_AQI_Weather['hour'] = dataset_AQI_Weather['Datetime'].apply(lambda x: sin_cos(x[11:13],mode = 'hour'))
dataset_AQI_Weather['day'] = dataset_AQI_Weather['Datetime'].apply(lambda x: sin_cos(x[8:10], mode = 'day'))
dataset_AQI_Weather['month'] = dataset_AQI_Weather['Datetime'].apply(lambda x: sin_cos(x[5:7], mode = 'month'))

#apply deg2rad to columns 'winddirDegree'
dataset_AQI_Weather['winddirDegree'] = dataset_AQI_Weather['winddirDegree'].apply(deg2rad)

#drop Datetime column
dataset_AQI_Weather.drop('Datetime', axis=1, inplace=True)

#use minmax scaler on dataset_AQI_Weather
dataset_AQI_Weather = pd.DataFrame(scaler.fit_transform(dataset_AQI_Weather), columns=dataset_AQI_Weather.columns, index=dataset_AQI_Weather.index)

#make 'AQI' as the target variable
dataset_y = dataset_AQI_Weather['AQI']
dataset_x = dataset_AQI_Weather#.drop(['AQI', 'PM2.5', 'PM10'], axis=1)

#shift dataset_y by 24
dataset_y = dataset_y.shift(-24)

#drop the last 24 rows of dataset_x and dataset_y
dataset_x = dataset_x.drop(dataset_x.index[-24:])
dataset_y = dataset_y.drop(dataset_y.index[-24:])

#make dataframe dataset_test where index contains '2019'
dataset_test_y = dataset_y[dataset_y.index.str.contains('2019')]
dataset_test_x = dataset_x[dataset_x.index.str.contains('2019')]

#remove rows where '2019' is in index in dataset_x and dataset_y
dataset_y = dataset_y[~dataset_y.index.str.contains('2019')]
dataset_x = dataset_x[~dataset_x.index.str.contains('2019')]

In [11]:
#write a function to train a neural network on dataset_x and dataset_y
def train_model(dataset_x, dataset_y, dataset_test_x):
    #create a neural network
    model = Sequential()
    #add a layer with 100 neurons
    model.add(Dense(100, input_dim=dataset_x.shape[1], activation='relu'))
    #add a layer with 50 neurons
    model.add(Dense(50, activation='relu'))
    #add a layer with 25 neurons
    model.add(Dense(25, activation='relu'))
    #add a layer with 10 neurons
    model.add(Dense(10, activation='relu'))
    #add a layer with 1 neuron
    model.add(Dense(1, activation='linear'))
    #compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    #fit the model on the training set
    model.fit(dataset_x, dataset_y, epochs=10, batch_size=1, verbose=1)
    #make predictions on the test set
    predictions = model.predict(dataset_test_x)
    #return the model
    return model, predictions

In [96]:
model, predictions = train_model(dataset_x, dataset_y, dataset_test_x)
#save the model
model.save('model.h5')

In [13]:
predictions = predictions.reshape(predictions.shape[0])
plt.figure(figsize=(100,20))
plt_x = dataset_test_x.index.values
plt.plot(plt_x, predictions, c='red', label = 'predictions')
plt.plot(plt_x, dataset_test_y, c='blue', label = 'ground truth')
plt.xticks(rotation=90)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

In [14]:
#calculate the accuracy of the predictions against dataset_test_y
accuracy = mean_squared_error(dataset_test_y, predictions)
print(accuracy)